In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime



In [2]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [3]:
df_jtc = pd.read_csv('allvol_jtc.csv')
dd = []
        
        
ds = [d.split('/') for d in df_jtc['date'].values]
d2 = [f"{d[0]}/{d[1]}/{(1900 if int(str(d[2])) > 20 else 2000) + int(str(d[2]))}" for d in ds]
d3 = [datetime.datetime.strptime(d,'%m/%d/%Y')  for d in d2]

df_jtc2 = df_jtc.copy()
df_jtc2['dt_date'] = d3

df_jtc2 = df_jtc2.copy()
df_jtc2['year'] = df_jtc2.dt_date.dt.year
df_jtc2['month'] = df_jtc2.dt_date.dt.month
df_jtc2['week'] = df_jtc2.dt_date.dt.week
df_jtc2['yyyyww'] = df_jtc2.year * 100 + df_jtc2.week
df_jtc2_weekly = df_jtc2[['yyyyww','vol']].groupby(['yyyyww'],as_index=False).max()
df_jtc2_weekly.yyyyww = df_jtc2_weekly.yyyyww.apply(lambda yyyyww:f"{str(yyyyww)[0:4]}-W{str(yyyyww)[-2:]}")
df_jtc2_weekly.yyyyww = df_jtc2_weekly.yyyyww.apply(lambda d:datetime.datetime.strptime(d + '-1', "%Y-W%W-%w"))
df_jtc2_weekly.yyyyww = df_jtc2_weekly.yyyyww.apply(lambda d:d.strftime('%m/%d/%Y'))

df_jtc2['yyyymm'] = df_jtc2.year.astype(str) + "-" + df_jtc2.month.astype(str)
df_jtc2_monthly = df_jtc2[['yyyymm','vol']].groupby(['yyyymm'],as_index=False).max()


In [4]:
l=r=b=t=50
p = 20
barplot = False
f1 = plotly_plot(df_in=df_jtc,x_column='date',bar_plot=barplot,y_left_label="Implied Volatility",
                  plot_title="Crude Oil Daily Implied Volatility")
f1.update_layout(margin=dict(l=l,r=r,b=b,t=t,pad=p))
f2 = plotly_plot(df_in=df_jtc2_weekly,x_column='yyyyww',bar_plot=barplot,y_left_label="Implied Volatility",
                  plot_title="Crude Oil Weekly High Implied Volatility")
f2.update_layout(margin=dict(l=l,r=r,b=b,t=t,pad=p))
f3= plotly_plot(df_in=df_jtc2_monthly,x_column='yyyymm',bar_plot=barplot,y_left_label="Implied Volatility",
                  plot_title="Crude Oil Monthly High Implied Volatility")
f3.update_layout(margin=dict(l=l,r=r,b=b,t=t,pad=p))

iplot(f1)
iplot(f2)
iplot(f3)
